In [2]:
import comtradeapicall
import json
import pandas as pd
import time

In [70]:
key = '79941b0b7060417b83fc3d4a4813e193'

In [6]:
#country list
with open("../data/countries.json", "r", encoding="utf-8") as f:
    data = json.load(f)


In [7]:
country_df = pd.DataFrame(data['results'])


In [8]:
country_df.head()

,id,text,reporterCode,reporterDesc,reporterNote,reporterCodeIsoAlpha2,reporterCodeIsoAlpha3,entryEffectiveDate,isGroup,entryExpiredDate
0,4,Afghanistan,4,Afghanistan,Afghanistan,AF,AFG,1900-01-01T00:00:00,False,NaN
1,8,Albania,8,Albania,Albania,AL,ALB,1900-01-01T00:00:00,False,NaN
2,12,Algeria,12,Algeria,Algeria,DZ,DZA,1900-01-01T00:00:00,False,NaN
3,20,Andorra,20,Andorra,Andorra,AD,AND,1900-01-01T00:00:00,False,NaN
4,24,Angola,24,Angola,Angola,AO,AGO,1900-01-01T00:00:00,False,NaN


In [9]:
country_list = country_df.loc[country_df["isGroup"] == False, "id"].tolist()
country_list = [str(country) for country in country_list]
all_country = ",".join(country_list)

In [10]:
metal_list = ["8101", "8102", "8103", "8104", "8105", "8106", "8108", "8109", "8110", "8111", "7106", "7108", "7110"]

In [21]:
metal_dic = {
    "8101":"Tungsten",
    "8102":"Molybdenum",
    "8103":"Tantalum", 
    "8104":"Magnesium", 
    "8105":"Cobalt",
    "8106":"Bismuth",
    "8108":"Titanium",
    "8109":"Zirconium",
    "8110":"Antimony", 
    "8111":"Manganese",
    "7106":"Silver", 
    "7108":"Gold", 
    "7110":"Platinum"
}

In [68]:
df = comtradeapicall.previewFinalData(
    typeCode='C',
    freqCode='A',
    clCode='HS',
    reporterCode=all_country,
    partnerCode=all_country,
    partner2Code=0,
    customsCode=None, 
    motCode='0',
    period='2024',
    cmdCode='8101',
    flowCode='X'
)

print(df.head())

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3959506130.py, line 11)

In [56]:
#df.to_csv("../data/sample_all_to_all_8101.csv")

In [78]:
# df_list = []

# for commodity in metal_list:
#     time.sleep(0.5)
#     df = comtradeapicall.getFinalData(
#         subscription_key=key,
#         typeCode='C',
#         freqCode='A',
#         clCode='HS',
#         reporterCode=all_country,
#         partnerCode=all_country,
#         partner2Code=0,
#         customsCode=None,
#         motCode='0',
#         period='2024',
#         cmdCode=commodity,
#         flowCode='X'
#     )
#     if not df.empty:
#         df_list.append(df)


In [79]:
# trade_df = pd.concat(df_list, ignore_index=True)
# trade_df.to_csv("../data/trade2024.csv")

In [4]:
getting_year = ['2022', '2023', '2024']
csv_files = [f'../data/trade{year}.csv' for year in getting_year]
trade_dfs = [pd.read_csv(f) for f in csv_files]
combined_trade_df = pd.concat(trade_dfs, ignore_index=True)

In [33]:
combined_trade_df['cmdCode'] = combined_trade_df['cmdCode'].astype('str')
combined_trade_df['period'] = combined_trade_df['period'].astype('str')

In [34]:
combined_trade_df['source'] = combined_trade_df.apply(lambda x: country_df.loc[country_df['id'] == x['reporterCode'], 'text'].iloc[0], axis=1)
combined_trade_df['target'] = combined_trade_df.apply(lambda x: country_df.loc[country_df['id'] == x['partnerCode'], 'text'].iloc[0], axis=1)
combined_trade_df['product'] = combined_trade_df.apply(lambda x: metal_dic[x['cmdCode']], axis=1)

In [39]:
complete_df = combined_trade_df[['period', 'product', 'source', 'target', 'primaryValue']].rename(columns = {
    'period':'year',
    'primaryValue': 'value'
    })

In [41]:
complete_df

,year,product,source,target,value
0,2022,Tungsten,Australia,Fiji,9.343370e+02
1,2022,Tungsten,Australia,Ireland,1.872142e+03
2,2022,Tungsten,Australia,Italy,3.329485e+04
3,2022,Tungsten,Australia,Japan,3.205142e+05
4,2022,Tungsten,Australia,Malaysia,1.961150e+05
...,...,...,...,...,...
50611,2024,Platinum,Türkiye,South Africa,1.201000e+03
50612,2024,Platinum,Türkiye,Switzerland,4.532209e+06
50613,2024,Platinum,Türkiye,United Kingdom,3.719000e+03
50614,2024,Platinum,Türkiye,USA,3.549010e+05


In [38]:
complete_df.to_csv('../data/tradedata.csv')

In [40]:
complete_df['year'].dtypes

dtype('O')